In [7]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, timedelta
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import requests
import io
import time
import random
from urllib.error import HTTPError
import numpy as np

In [8]:
start_date = pd.to_datetime("2019-02-14")
end_date = pd.to_datetime("2025-02-14")
stock_data = yf.download(
                'NVDA', 
                start=start_date, 
                end=end_date + timedelta(days=1),
                progress=False  # Disable progress bar to reduce console output
            )

In [9]:
def generate_demo_data(ticker, start_date, end_date):
    """Generate synthetic stock data for demo purposes"""
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # Business days
    
    # Generate random price data
    np.random.seed(hash(ticker) % 10000)  # Use ticker as seed for consistency
    
    # Start with a base price based on the ticker
    base_price = 100 + hash(ticker) % 900  # Between 100 and 1000
    
    # Generate daily returns with some randomness
    daily_returns = np.random.normal(0.0005, 0.015, size=len(date_range))
    
    # Calculate cumulative returns
    cumulative_returns = np.cumprod(1 + daily_returns)
    
    # Calculate prices
    prices = base_price * cumulative_returns
    
    # Create a dataframe
    data = {
        'Open': prices * np.random.uniform(0.99, 1.0, size=len(date_range)),
        'High': prices * np.random.uniform(1.0, 1.03, size=len(date_range)),
        'Low': prices * np.random.uniform(0.97, 0.99, size=len(date_range)),
        'Close': prices,
        'Volume': np.random.randint(1000000, 10000000, size=len(date_range))
    }
    
    df = pd.DataFrame(data, index=date_range)
    return df
df = generate_demo_data('NVDA', start_date, end_date)

In [14]:
print(type(df.index))
df.head()

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,Open,High,Low,Close,Volume
2019-02-14,722.146151,734.347046,713.969929,725.285381,2151646
2019-02-15,736.257483,745.411800,733.477147,743.495460,4576291
2019-02-18,733.133130,753.150065,721.716346,738.539927,1296667
2019-02-19,732.573194,744.394697,719.116627,736.806929,8956474
2019-02-20,724.574776,740.124847,720.227149,727.874697,7424439


In [13]:
print(type(stock_data.index))
stock_data.head()

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2019-02-14,3.831803,3.860071,3.746751,3.789897,827264000
2019-02-15,3.901481,4.063402,3.878668,4.040589,1516748000
2019-02-19,3.884124,3.964713,3.871726,3.890819,551892000
2019-02-20,3.931486,3.998684,3.902226,3.913384,540988000
2019-02-21,3.862551,3.968680,3.847921,3.944132,448548000


In [17]:
fig = make_subplots(
        rows=len([1]), 
        cols=1,
    )
# fig.add_trace(
#                 go.Line(
#                     x=stock_data.index,
#                     y=stock_data['Close'].values.flatten(),
#                     name='ticker',
#                     line=dict(width=2),
#                     mode='lines',  # Explicitly set mode to lines
#                     opacity=0.8    # Make slightly transparent to see markers better
#                 ),
#                 row=1, col=1
#             )
fig.add_trace(
                go.Candlestick(
                    x=stock_data.index,
                    open=stock_data['Open'].values.flatten(),
                    high=stock_data['High'].values.flatten()    ,
                    low=stock_data['Low'].values.flatten(),
                    close=stock_data['Close'].values.flatten(),
                    name='ticker',
                    increasing=dict(line=dict(color='#26a69a')),
                    decreasing=dict(line=dict(color='#ef5350'))
                ),
                row=1, col=1
            )

In [3]:
# Create a simple Dash plot of stock data
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Create a simple plotly figure
fig = px.line(
    x=stock_data.index, 
    y=stock_data['Close'].values.flatten(),
    # labels={'x': 'Date', 'y': 'Stock Price'},
    # title='NVIDIA Stock Price Over Time'
)

# Add some styling
fig.update_layout(
    template='plotly_white',
    xaxis=dict(
        title='Date',
        gridcolor='lightgray',
        showgrid=True
    ),
    yaxis=dict(
        title='Price (USD)',
        gridcolor='lightgray',
        showgrid=True
    ),
    hovermode='x unified'
)

# Display the figure
fig.show()

# # Alternative simple matplotlib plot
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 6))
# plt.plot(stock_data.index, stock_data['Close'])
# plt.title('NVIDIA Stock Price')
# plt.xlabel('Date')
# plt.ylabel('Price (USD)')
# plt.grid(True, alpha=0.3)
# plt.tight_layout()
# plt.show()
